In [ ]:
from google.colab import drive
drive.mount('/content/drive2', force_remount=True)
driveHome = "/content/drive2/MyDrive/MasterThesis"

Mounted at /content/drive2


In [ ]:
def flushDrive():
    drive.flush_and_unmount()
    drive.mount('/content/drive2', force_remount=True)

In [ ]:
!python -m pip install git+https://github.com/kinshuk-h/pytube
!pip install opencv-python-headless==4.5.4.60

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/kinshuk-h/pytube to /tmp/pip-req-build-krryi_cx
  Running command git clone -q https://github.com/kinshuk-h/pytube /tmp/pip-req-build-krryi_cx
  Created wheel for pytube: filename=pytube-12.1.0-py3-none-any.whl size=56522 sha256=b0442928ccf54927d65458f55841afd0cb7dad775a31950d76d1c8b85e5b3f54
  Stored in directory: /tmp/pip-ephem-wheel-cache-btydiej1/wheels/42/db/9b/1e0377f281a659b3f4d00c820d69fa362b93f267c410aeff4f
Successfully built pytube
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.6 MB 76 kB/s 


In [ ]:
import os
from os.path import join, basename
from pathlib import Path
import moviepy.editor
import cv2
import shutil
import time
import re
from collections import defaultdict

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2154496/45929032 bytes (4.7%)5087232/45929032 bytes (11.1%)7815168/45929032 bytes (17.0%)10256384/45929032 bytes (22.3%)12787712/45929032 bytes (27.8%)15556608/45929032 bytes (33.9%)18423808/45929032 bytes (40.1%)21544960/45929032 bytes (46.9%)24281088/45929032 bytes (52.9%)27148288/45929032 bytes (59.1%)30015488/45929032 bytes (65.4%)32997376/45929032 bytes (71.8%)

In [ ]:
Path(driveHome).mkdir(parents=True, exist_ok=True)

In [ ]:
classes = [
           "BlowingCandles",
           "Billiards", 
           "BrushingTeeth", 
           "CuttingInKitchen",
           "Mixing",
           "MoppingFloor", 
           "ShavingBeard", 
           "Swing",
           "Typing",
           "WallPushups",
]

In [ ]:
%%capture
!wget --no-check-certificate https://www.crcv.ucf.edu/datasets/human-actions/ucf101/UCF101.rar
shutil.move("UCF101.rar", driveHome)
!unrar x "/content/drive/MyDrive/MasterThesis/UCF101.rar" $driveHome

In [ ]:
if not os.path.exists(os.path.join(driveHome, "UCF-101")):
    shutil.copytree("UCF-101", os.path.join(driveHome, "UCF-101"))

In [ ]:
def groupFrames(pathToSource, pathToTarget, quantity):
    if os.path.exists(pathToTarget):
        print("Skipping")
        return
    Path(pathToTarget).mkdir(parents=True)
    pictureCount = len(os.listdir(pathToSource))
    skip = pictureCount // quantity
    if skip < 1:
        skip = 1
    copied = 0
    for i, picture in enumerate(sorted(os.scandir(pathToSource), key = lambda n: n.name), 1):
        if i % skip == 0 and copied < quantity:
            shutil.copyfile(picture.path, os.path.join(pathToTarget, os.path.basename(picture.path)))
            copied += 1
        else:
            continue

In [ ]:
def split_frames_by_groups(path_to_source, path_to_target, quantity, max_groups_fraction):
    if os.path.exists(path_to_target):
        print("Skipping")
        return
    Path(path_to_target).mkdir(parents=True)

    frames_groups = defaultdict(list)
    #podział plików na grupy
    for frame in os.scandir(path_to_source):
        frame_path = frame.path
        # grupy w plikach mają postać np. _g02_
        found = re.search(r"_g\d{2}_", frame_path)
        group = found.group()[2:4]
        frames_groups[group].append(frame_path)

    available_groups = len(frames_groups)
    chosenGroups = int(max_groups_fraction * available_groups)

    # wybranie określonego procenta grup
    frames_from_chosen_groups = []
    [frames_from_chosen_groups.extend(paths) for group, paths in frames_groups.items() if int(group) <= chosenGroups]

    picture_count = len(frames_from_chosen_groups)
    skip = picture_count // quantity
    if skip < 1:
        skip = 1
    copied = 0
    for i, picture in enumerate(sorted(frames_from_chosen_groups), 1):
        if i % skip == 0 and copied < quantity:
            shutil.copyfile(picture, os.path.join(path_to_target, basename(picture)))
            copied += 1
        else:
            continue

In [ ]:
def videosToFrames(starting_path):
    videoNr = 0
    for video in os.scandir(starting_path):
        if str(video.name).endswith(".avi"):
            frameNr = 0
            capture = cv2.VideoCapture(video.path)
            while (True):
                success, frame = capture.read()
                if success:
                    frameName = os.path.join(starting_path, f"title_{video.name}_fr_{frameNr:04}.jpg")
                    cv2.imwrite(frameName, frame)
                else:
                    break
                frameNr += 1
            
            videoNr += 1
            capture.release()
            os.remove(video.path)

In [ ]:
def split_ucf_movies(source, dst, quantity=[]):
    if len(quantity) != 3 or sum(quantity) != 100:
        print("Wrong quantities")
        return
    dstTrain = os.path.join(dst, "train")
    dstTest = os.path.join(dst, "test")
    dstExam = os.path.join(dst, "exam")
    Path(dst).mkdir(parents=True, exist_ok=True)
    Path(dstTrain).mkdir(exist_ok=True)
    Path(dstTest).mkdir(exist_ok=True)
    Path(dstExam).mkdir(exist_ok=True)

    for c in classes:
        pathToMovies = os.path.join(source, c)
        print(pathToMovies)

        movieGroups = defaultdict(list)
        for movie in os.listdir(pathToMovies):
            moviePath = join(pathToMovies, movie)
            found = re.search(r"_g\d{2}_", moviePath)
            group = found.group()[2:4]
            movieGroups[group].append(moviePath)

        def helper(dst, quantity):
            Path(os.path.join(dst, c)).mkdir(exist_ok=True)
            if len(os.listdir(os.path.join(dst,c))) > 0:
                return
            counter = 0
            movedGroups = []
            for key, value in movieGroups.items():
                movedGroups.append(key)
                counter += len(value)
                for v in value:
                    shutil.copyfile(v, os.path.join(dst, c, os.path.basename(v)))
                if counter >= quantity:
                    break
            for group in movedGroups:
                movieGroups.pop(group)

        helper(dstTrain, quantity[0])
        helper(dstTest, quantity[1])
        helper(dstExam, quantity[2])

In [ ]:
UcfPath = os.path.join(driveHome, "UCF-SmartSplit")
UcfConstantValidation = os.path.join(UcfPath, "validation-constant")
UcfChosenVideos = os.path.join(UcfPath, "videos_or_frames")
UcfFrames = os.path.join(UcfPath, "frames")
UcfAugmentExtract = os.path.join(UcfPath, "augmentation-prep")

In [ ]:
split_ucf_movies(os.path.join(driveHome, "UCF-101"), UcfChosenVideos, [70, 20, 10])
flushDrive()

In [ ]:
quantities = [250, 500, 750, 1000, 1500, 2000, 3000]

In [ ]:
savedVideos = os.path.join(UcfChosenVideos, "train")
for actionType in os.scandir(savedVideos):
    print(f"\n------{actionType.name}------")
    pathToSourceVideos = os.path.join(savedVideos, actionType.name)
    videosToFrames(pathToSourceVideos)
    for quantity in quantities:
        pathToTarget = os.path.join(UcfFrames, str(quantity), "train", actionType.name)
        print(pathToTarget)
        split_frames_by_groups(pathToSourceVideos, pathToTarget, quantity, quantity/max(quantities))

flushDrive()


------BlowingCandles------
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/250/train/BlowingCandles
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/500/train/BlowingCandles
Skipping
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/750/train/BlowingCandles
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/1000/train/BlowingCandles
Skipping
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/1500/train/BlowingCandles
Skipping
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/2000/train/BlowingCandles
Skipping
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/3000/train/BlowingCandles
Skipping

------Billiards------
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/250/train/Billiards
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/500/train/Billiards
Skipping
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/750/train/Billiards
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/frames/1000/tr

In [ ]:
savedVideos = os.path.join(UcfChosenVideos, "test")
for actionType in os.scandir(savedVideos):
    print(f"\n------{actionType.name}------")
    pathToSourceVideos = os.path.join(savedVideos, actionType.name)
    videosToFrames(pathToSourceVideos)
    quantity = max(quantities)
    pathToTarget = os.path.join(UcfConstantValidation, actionType.name)
    groupFrames(pathToSourceVideos, pathToTarget, quantity * 0.2)

flushDrive()


------BlowingCandles------
Skipping

------Billiards------
Skipping

------BrushingTeeth------
Skipping

------CuttingInKitchen------
Skipping

------Mixing------
Skipping

------MoppingFloor------
Skipping

------ShavingBeard------
Skipping

------Swing------
Skipping

------Typing------
Skipping

------WallPushups------
Skipping
Mounted at /content/drive2


In [ ]:
savedVideos = os.path.join(UcfChosenVideos, "train")
percentages = [0.1, 0.333, 0.666, 0.9]
for p in percentages:
    for actionType in os.listdir(savedVideos):
        print(f"\n------{actionType}------")
        pathToSourceVideos = os.path.join(savedVideos, actionType)
        for quantity in quantities:
            pathToTarget = os.path.join(UcfAugmentExtract, str(p), str(quantity), actionType)
            print(pathToTarget)
            split_frames_by_groups(pathToSourceVideos, pathToTarget, quantity * p, quantity/max(quantities))
    
    flushDrive()


------BlowingCandles------
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/250/BlowingCandles
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/500/BlowingCandles
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/750/BlowingCandles
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/1000/BlowingCandles
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/1500/BlowingCandles
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/2000/BlowingCandles
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/3000/BlowingCandles

------Billiards------
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/250/Billiards
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/500/Billiards
/content/drive2/MyDrive/MasterThesis/UCF-SmartSplit/augmentation-prep/0.1/750/Billiards
/content/drive2/MyDrive/Master

In [ ]:
flushDrive()

Mounted at /content/drive2
